In [1]:
import pandas as pd
from df_transform import *
from statistical_tests import *

### Loading in and cleaning the dataset

In [10]:
survey = pd.read_csv("../data/cookie_survey_responses.csv")
survey.drop([c for c in survey.columns[:2]], axis=1, inplace=True)
new_df, question_map = rename_columns(survey)
new_df = new_df.drop(0)
new_df["Question 1"] = new_df["Question 1"].astype(int)
new_df["Question 13"] = new_df["Question 13"].astype(int)
answer_encs = encode_answers(new_df)
N = len(new_df)  # number of observations
print(new_df.shape)
print(new_df.columns)
new_df.head(5)

(22, 22)
Index(['Question 1', 'Question 2', 'Question 3', 'Question 4', 'Question 5',
       'Question 6', 'Question 7', 'Question 8', 'Question 9', 'Question 10',
       'Question 11', 'Question 12', 'Question 13', 'Question 14',
       'Question 15', 'Question 16', 'Question 17', 'Question 18',
       'Question 19', 'Question 20', 'Question 21', 'Question 22'],
      dtype='object')


,Question 1,Question 2,Question 3,Question 4,Question 5,Question 6,Question 7,Question 8,Question 9,Question 10,...,Question 13,Question 14,Question 15,Question 16,Question 17,Question 18,Question 19,Question 20,Question 21,Question 22
1,20,Economics & Data Science,Safari,2,0,2,4,2,1,1,...,4,Not sure,Yes,"Generally shopping/marketplace websites, e.g. ...",No,NaN,0,1,1,1
2,21,Linguistics,Chrome,2,0,2,4,0,1,0,...,4,No,No,NaN,No,NaN,0,0,0,5
3,19,Chemistry,Chrome,2,0,1,5,0,1,0,...,2,No,No,NaN,No,NaN,2,0,0,3
4,34,Biophysics,Chrome,0,2,0,0,2,2,0,...,3,Not sure,I didn't know that was possible,NaN,No,NaN,0,0,0,9
5,21,Accounting,Chrome,2,0,2,4,1,3,2,...,4,Not sure,I didn't know that was possible,NaN,Yes,I did not the website to have my data,0,1,1,2


Adding a expertise categorization column (STEM or Non STEM)

In [11]:
# 1 means STEM, 0 means non-stem. Obtained via manual observation
stem_cat = pd.Series([1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0])
# 1 means directly related to computers/data, 0 means not computer/data related
comp_cat = pd.Series([1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0])
new_df["stem"] = stem_cat
new_df["computers"] = comp_cat

### Run Statistical Tests

Compare number of adults that claim a comprehensive understanding of cookies in our study (22 observations) to a Statista survey of adults (1100 observations). <br/> 
Corresponds to Question 5: <br/> 

Do you understand what a cookie does and what data you give away when you accept cookies?
{0: 'I understand the basics',
 1: 'No, not at all',
 2: 'Not really',
 3: 'Yes, fully'}

In [47]:
# those who have comprehensive understanding
p_exp = len(new_df[new_df["Question 5"] == 3]) / len(new_df)  # 4/22
p_comp = .28  # see cookieeyes.com link in cookie_awareness.md
print("Comprehensive Knowledge:", two_prop_z_test(p_exp, p_comp, N, 1100))
# partial knowledge
p_exp = len(new_df[new_df["Question 5"] == 0]) / len(new_df)
p_comp = .49  # see cookieeyes.com link in cookie_awareness.md
print("Partial Knowledge:", two_prop_z_test(p_exp, p_comp, N, 1100))
# no knowledge
p_exp = len(new_df.loc[(new_df["Question 5"] == 1) | (new_df["Question 5"] == 2)]) / len(new_df)
p_comp = .24  # see cookieeyes.com link in cookie_awareness.md
print("No Knowledge:", two_prop_z_test(p_exp, p_comp, N, 1100))

Comprehensive Knowledge: (-1.0176896912577567, 0.3088254478393895, 'fail to reject')
Partial Knowledge: (-0.32939162299787755, 0.7418596987241401, 'fail to reject')
No Knowledge: (1.3398507379555382, 0.18029387638173944, 'fail to reject')


Next, we check to see if there's any significance in the proportions of students that blindly accept cookies. We compare to the allaboutcookies study of 1000 observations of US adults which found that 38% of adults do so. We are interested in option "0" for Question 11: <br/> 

When you visit a website and see a cookie consent banner, what do you usually do?
{0: 'Accept all cookies without reading', 1: 'Decline non-essential cookies', 2: 'Leave the website'}
<br/> 

Then we isolate the Gen Z respondents by age (<= 28) and compare the proportion that blindly accept all cookies. In the allaboutcookies article, this number is .47.

In [48]:
# blindly accept whole sample
p_exp = len(new_df.loc[new_df["Question 11"] == 0]) / len(new_df)
p_comp = .38  # see allaboutcookies.org link in cookie_awareness.md
print("Whole Sample:", two_prop_z_test(p_exp, p_comp, N, 1000))
# blindly accept for gen Z
p_exp = len(new_df.loc[(new_df["Question 1"] <= 28) & (new_df["Question 11"] == 0)]) / len(new_df)
n_exp = sum(new_df["Question 1"] <= 28)
p_comp = .47  # see allaboutcookies.org link in cookie_awareness.md
print("Gen Z Only:", two_prop_z_test(p_exp, p_comp, n_exp, 250))

Whole Sample: (-0.15644299315234023, 0.8756838507100282, 'fail to reject')
Gen Z Only: (-1.3111114469863319, 0.18982011394345655, 'fail to reject')


### Computer vs Non-Computer Expertise Statistical Tests

In [48]:
n_comp = len(new_df.loc[new_df["computers"] == 1])
n_non_comp = len(new_df.loc[new_df["computers"] == 0])

Compare the proportions that selected "Yes, fully" or "I understand the basics". <br/> 

Do you understand what a cookie does and what data you give away when you accept cookies?
{0: 'I understand the basics', 1: 'No, not at all', 2: 'Not really', 3: 'Yes, fully'}

In [50]:
p_comp = len(new_df.loc[(new_df["computers"] == 1) & ((new_df["Question 5"] == 0) | (new_df["Question 5"] == 3))]) / n_comp
p_non_comp = len(new_df.loc[(new_df["computers"] == 0) & ((new_df["Question 5"] == 0) | (new_df["Question 5"] == 3))]) / n_non_comp
print("Computer majors understand cookies:", two_prop_z_test(p_comp, p_non_comp, n_comp, n_non_comp))

Computer majors understand cookies: (-1.636634176769943, 0.10170693003109332, 'fail to reject')


Compare the proportions that selected "Rarely" or "Never" accept all cookies. <br/> 

How often do you accept all cookies without changing any settings?

In [51]:
p_comp = len(new_df.loc[(new_df["computers"] == 1) & ((new_df["Question 12"] == "Rarely") | (new_df["Question 12"] == "Never"))]) / n_comp
p_non_comp = len(new_df.loc[(new_df["computers"] == 0) & ((new_df["Question 12"] == "Rarely") | (new_df["Question 5"] == "Never"))]) / n_non_comp
print("Computer majors accept cookies:", two_prop_z_test(p_comp, p_non_comp, n_comp, n_non_comp))

Computer majors accept cookies: (1.3228756555322951, 0.18587673236587587, 'fail to reject')


Compare the means for level of ease to reject cookies as well. <br/> 

How easy, on a scale of 1–5, do you find it to reject cookies that you don't want used?

In [52]:
mu_comp = new_df.loc[new_df["computers"] == 1]["Question 13"].mean()
std_comp = new_df.loc[new_df["computers"] == 1]["Question 13"].std()
mu_non_comp = new_df.loc[new_df["computers"] == 0]["Question 13"].mean()
std_non_comp = new_df.loc[new_df["computers"] == 0]["Question 13"].std()
print("Level of Ease to reject cookies:", two_sample_t_test(mu_comp, mu_non_comp, std_comp, std_non_comp, n_comp, n_non_comp))

Level of Ease to reject cookies: (2.3569540848602757, 0.03516316424650312, 'reject')


Compare the same means but on the basis of browser.

Brave vs Others

In [12]:
browsers = new_df["Question 3"].value_counts()
mu_comp = new_df.loc[new_df["Question 3"] == "Brave"]["Question 13"].mean()
std_comp = new_df.loc[new_df["Question 3"] == "Brave"]["Question 13"].std()
mu_non_comp = new_df.loc[new_df["Question 3"] == "Safari"]["Question 13"].mean()
std_non_comp = new_df.loc[new_df["Question 3"] == "Safari"]["Question 13"].std()
print("Level of Ease to reject cookies:", two_sample_t_test(mu_comp, mu_non_comp, std_comp, std_non_comp, browsers["Brave"], browsers["Safari"]))

Level of Ease to reject cookies: (0.46625240412015634, 0.6769990551482545, 'fail to reject')


In [13]:
browsers = new_df["Question 3"].value_counts()
mu_comp = new_df.loc[new_df["Question 3"] == "Brave"]["Question 13"].mean()
std_comp = new_df.loc[new_df["Question 3"] == "Brave"]["Question 13"].std()
mu_non_comp = new_df.loc[new_df["Question 3"] == "Chrome"]["Question 13"].mean()
std_non_comp = new_df.loc[new_df["Question 3"] == "Chrome"]["Question 13"].std()
print("Level of Ease to reject cookies:", two_sample_t_test(mu_comp, mu_non_comp, std_comp, std_non_comp, browsers["Brave"], browsers["Chrome"]))

Level of Ease to reject cookies: (1.7469367235686857, 0.19120787377273207, 'fail to reject')
